## Problem Statement
An air compressor has an isentropic efficiency of 80% and operates in a steady-state, steady-flow (SSSF) configuration with negligible changes in kinetic and potential energy.  It receives a volumetric flow rate of 3000 CFM with an inlet pressure of $p_1=14.7\,\text{psia}$ and inlet temperature of $T_1=70^\circ\text{F}$.  It compresses the air by a factor of 10. Determine<br>
(a) Rate of compressor work, $\mathrm{HP}$<br>
(b) Rate of entropy generation, $\mathrm{Btu/min^\circ R}$

## Solution

__[Video Explanation](https://uky.yuja.com/V/Video?v=3071282&node=10458660&a=1537564096&autoplay=1)__

In the previous example (Ex 0.1), we wrote two separate python scripts to illustrate the difference between using only the standard python library and using third-party modules.  Those two scripts were placed in two self-contained code blocks to help show the separation between the two.  In this example we will jump straight to using the third-party libraries to make things easier, and we will spread the python code out across multiple cells with explanatory text (markdown) cells to describe the code rather than using python comments.  To execute this code you will need to execute each code block in order, or select the `Run All` option from the `Cell` menu above.

### Python Initialization
We'll start by importing the libraries we will use for our analysis.  

In [3]:
from kilojoule.templates.USCS_R import *

In the first version of the previous example we defined a new variable for each property at each state, but in this example (and future examples) we will instead store the values in a custom python data structure from `kilojoule`.  This is a different approach to variable naming and organization that will allow us to do some interesting things later on.

We can use the `kilojoule` library to look up property values for real fluid using the `realfluid.Properties()` class or for ideal gasses using the `idealgas.Properties()`.  For each of these functions you initialize (instantiate) the class by passing it the name of the fluid and an optional preferred unit system to use when returning values (default: SI in $\mathrm{^\circ C}$).  For this case, we will treat the air as an ideal gas, so we will use the `idealgas.Properties()` class.

In [4]:
air = idealgas.Properties('Air')

### Given Parameters
We now define variables to hold our known values.

In [5]:
%%showcalc
# the next three lines show different ways to create a dimensional quantity, but the Quantity(magnitude,"units") syntax is the preferred method.
T[1] = Quantity(70.,'degF')        # inlet temperature
p[1] = 14.7*ureg.psi               # inlet pressure
Vdot[1] = 3000.0*ureg('ft^3/min')  # volumetric flow rate at inlet
p[2] = 10*p[1]                     # exit pressure
eta_c = Quantity(0.8,'')           # isentropic efficiency

Summary();

\begin{align}
  {T}_{ 1 } &=  70.0\ \mathrm{°F}  = 529.67\ \mathrm{°R} 
\end{align}

\begin{align}
  {p}_{ 1 } &=   14.7  \cdot  \underset{ ureg }{ psi }  \\
    &=  14.7  \cdot   pound_force_per_square_inch  \\
    &= 14.7\ \mathrm{psi}
\end{align}

\begin{align}
  {\dot{V}}_{ 1 } &=   3000.0  \cdot  \mathrm{ ureg }\left( ft^3/min \right)   = 1.4158\ \frac{\mathrm{m}^{3}}{\mathrm{s}} 
\end{align}

\begin{align}
  {p}_{ 2 } &=   10  \cdot  {p}_{ 1 }  \\
    &=  10  \cdot  14.7\ \mathrm{psi} \\
    &= 147.0\ \mathrm{psi}
\end{align}

\begin{align}
  \eta_{c} &=  0.8 
\end{align}


\begin{align}{ \eta_{c} }&={ 0.8 }&{  }\end{align}
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>$T$ [°R]</th>
      <th>$p$ [psi]</th>
      <th>$\dot{V}$ [m³/s]</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>529.67</td>
      <td>14.7</td>
      <td>1.415842</td>
    </tr>
    <tr>
      <th>2</th>
      <td>-</td>
      <td>147.0</td>
      <td>-</td>
    </tr>
  </tbody>
</table>



### Assumptions
  - Negligible changes in kinetic energy 
  - Negligible changes in potential energy 
  - Adiabatic (no heat transfer)
  - Constant specific heat (cold-air-standard)
  - Ideal gas (cold-air-standard)

We could pull properties for air from the tables in the back of the book since we are assuming constant specific heat and ideal gas behavior, or we can look them up using the `air` reference we created earlier


In [6]:
%%showcalc 
"Ideal Gas"
R = air.R   # specific gas constant

"Constant thermal properties at room temperature"
T_room = Quantity(25,'degC') # room temperature
c_p = air.cp(T_room)         # constant pressure specific heat at room temperature
k = air.k(T_room)            # specific heat ratio at room temperature

Ideal Gas
\begin{align}
  R &=  \underset{ air }{ R }  = 0.28705\ \frac{\mathrm{kJ}}{\left(\mathrm{K} \cdot \mathrm{kg}\right)} 
\end{align}

Constant thermal properties at room temperature
\begin{align}
  T_{room} &=  25\ \mathrm{°C} 
\end{align}

\begin{align}
  c_{p} &=  \mathrm{ \underset{ air }{ cp } }\left( T_{room} \right)  = \mathrm{ \underset{ air }{ cp } }\left( 25\ \mathrm{°C} \right) = 1.0047\ \frac{\mathrm{kJ}}{\left(\mathrm{K} \cdot \mathrm{kg}\right)}
\end{align}

\begin{align}
  k &=  \mathrm{ \underset{ air }{ k } }\left( T_{room} \right)  = \mathrm{ \underset{ air }{ k } }\left( 25\ \mathrm{°C} \right) = 1.4
\end{align}



### Isentropic Efficiency
The isentropic efficiency of a compressor is defined as the ratio of the work that would be required if the compressor were ideal (isentropic) and operating between the same inlet state and exit pressure as the real device to the actual work.
$$\eta_c=\frac{\dot{W}_s}{\dot{W}_c}$$
where $\dot{W}_s$ is the rate of isentropic work and $\dot{W}_c$ is the rate of actual compressor work.  From a first law analysis, we can rewrite the work terms as changes in enthalpy between the inlet and exit states.
$$\require{cancel}
\eta_c = \frac{\cancel{\dot{m}}(h_{2s}-h_1)}{\cancel{\dot{m}}(h_2-h_1)}$$
Applying the constant specific heat assumption allows us to rewrite the changes in enthalpy as $\Delta h=c_p\Delta T$
$$\require{cancel}
\eta_c = \frac{\cancel{c_p}(T_{2s}-T_1)}{\cancel{c_p}(T_2-T_1)}$$
Our first goal is to find the exit temperature, so we solve for $T_2$
$$T_2 = T_1 + \frac{T_{2s}-T_1}{\eta_c}$$
However, in order to use this equation, we first need to find the temperature of the isentropic exit state, $T_{2s}$.  We can find this using ideal gas polytropic relations with $n=k$
$$T_{2s}=T_1\left(\frac{p_2}{p_1}\right)^{\frac{k-1}{k}}$$
Note that in order to apply the polytropic relation above, we must convert the temperatures to absolute units, i.e. $^\circ\text{R}$

In [7]:
%%showcalc
T['2s'] = T[1]*(p[2]/p[1])**((k-1)/k)
T[2] = T[1] + (T['2s']-T[1])/eta_c

\begin{align}
  {T}_{ {2s} } &=   {T}_{ 1 }  \cdot  {\left(\frac{ {p}_{ 2 } }{ {p}_{ 1 } }\right)}^{\frac{  k - 1  }{ k }}  \\
    &=  529.67\ \mathrm{°R}  \cdot  {\left(\frac{ 147.0\ \mathrm{psi} }{ 14.7\ \mathrm{psi} }\right)}^{\frac{  1.4 - 1  }{ 1.4 }} \\
    &= 1022.6\ \mathrm{°R}
\end{align}

\begin{align}
  {T}_{ 2 } &=  {T}_{ 1 } + \frac{  {T}_{ {2s} } - {T}_{ 1 }  }{ \eta_{c} } \\
    &= 529.67\ \mathrm{°R} + \frac{  1022.6\ \mathrm{°R} - 529.67\ \mathrm{°R}  }{ 0.8 }\\
    &= 1145.8\ \mathrm{°R}
\end{align}



Now that we know the actual exit temperature, we can find the actual rate of work using the 1st Law.
$$\dot{W}_c = \dot{m}c_p(T_2-T_1)$$
However, we will also need to use the ideal gas law to find the mass flow rate before applying this equation.
$$\dot{m}_1 = \frac{p_1\dot{V}_1}{RT_1}$$

In [8]:
%%showcalc
mdot = ((p[1]*Vdot[1])/(R*T[1].to('degR'))).to('lb/min')

Wdot_c = (mdot*c_p*(T[2]-T[1])).to('hp')


\begin{align}
  \dot{m} &=  \frac{  {p}_{ 1 }  \cdot  {\dot{V}}_{ 1 }  }{  R  \cdot  {T}_{ 1 }  } \\
    &= \frac{  14.7\ \mathrm{psi}  \cdot  1.4158\ \frac{\mathrm{m}^{3}}{\mathrm{s}}  }{  0.28705\ \frac{\mathrm{kJ}}{\left(\mathrm{K} \cdot \mathrm{kg}\right)}  \cdot  529.67\ \mathrm{°R}  }\\
    &= 224.72\ \frac{\mathrm{lb}}{\mathrm{min}}
\end{align}

\begin{align}
  \dot{W}_{c} &=    \dot{m}  \cdot  c_{p}   \cdot  \left(  {T}_{ 2 } - {T}_{ 1 }  \right)  \\
    &=   224.72\ \frac{\mathrm{lb}}{\mathrm{min}}  \cdot  1.0047\ \frac{\mathrm{kJ}}{\left(\mathrm{K} \cdot \mathrm{kg}\right)}   \cdot  \left(  1145.8\ \mathrm{°R} - 529.67\ \mathrm{°R}  \right) \\
    &= 783.55\ \mathrm{hp}
\end{align}



### Second Law Analysis
To determine the entropy generation, we need to do a 2nd Law analysis

$$\require{cancel}
\cancelto{0}{\frac{dS_{CV}}{dt}}= \sum_j\frac{\cancelto{0}{\dot{Q}_j}}{T_j}+\sum_i\dot{m}_is_i-\sum_e\dot{m}_es_e+\dot{S}_{gen}$$
$$\dot{S}_{gen} = \dot{m}(s_e-s_i)$$
which can be rewritten using the constant specific heat assumption as
$$\dot{S}_{gen} = \dot{m}\left[ c_p\ln\left(\frac{T_2}{T_1}\right)-R\ln\left(\frac{p_2}{p_1}\right)\right]$$
where the temperatures and pressures must be in absolute units for this equation to be valid.


In [9]:
%%showcalc
Sdot_gen = mdot*( c_p*ln(T[2]/T[1]) - R*ln(p[2]/p[1])) 

\begin{align}
  \dot{S}_{gen} &=   \dot{m}  \cdot  \left(   c_{p}  \cdot  \mathrm{ ln }\left( \frac{ {T}_{ 2 } }{ {T}_{ 1 } } \right)  -  R  \cdot  \mathrm{ ln }\left( \frac{ {p}_{ 2 } }{ {p}_{ 1 } } \right)   \right)  \\
    &=  224.72\ \frac{\mathrm{lb}}{\mathrm{min}}  \cdot  \left(   1.0047\ \frac{\mathrm{kJ}}{\left(\mathrm{K} \cdot \mathrm{kg}\right)}  \cdot  \mathrm{ ln }\left( \frac{ 1145.8\ \mathrm{°R} }{ 529.67\ \mathrm{°R} } \right)  -  0.28705\ \frac{\mathrm{kJ}}{\left(\mathrm{K} \cdot \mathrm{kg}\right)}  \cdot  \mathrm{ ln }\left( \frac{ 147.0\ \mathrm{psi} }{ 14.7\ \mathrm{psi} } \right)   \right) \\
    &= 25.69\ \frac{\mathrm{kJ} \cdot \mathrm{lb}}{\left(\mathrm{K} \cdot \mathrm{kg} \cdot \mathrm{min}\right)}
\end{align}



In [10]:
# Export your solution to submit to Canvas
# 1) Ensure all cells above show your final solution
# 2) Click the 'Save' button in the toolbar
# 3) Uncomment (remove the #) from the last line in this cell
# 4) Ensure the filename in the command below matches the filename of this notebook
# 4) Execute this cell to create an HTML file showing only the output 
# 5) Submit the .html file located in the folder with this notebook

!jupyter nbconvert --to html_embed --no-prompt 'Ex0.2 Compressor Isentropic Efficiency.ipynb'

[NbConvertApp] Converting notebook Ex0.2 Compressor Isentropic Efficiency.ipynb to html_embed


[NbConvertApp] Writing 595031 bytes to Ex0.2 Compressor Isentropic Efficiency.html
